# Data workflow - DEPRECATED

In [1]:
import pytorch_lightning 
import torch
import numpy as np
import matplotlib.pyplot as plt
import random
import sqlite3
from dataclasses import dataclass
import logging

import CPRD
from CPRD.data.foundational_loader import FoundationalDataModule

torch.manual_seed(1337)
logging.basicConfig(level=logging.INFO)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

# device = "cpu"    # just for debug errors

cuda


In [3]:
# Set GPT config to be equivalent
@dataclass
class DemoConfig:
    tokenizer: str = "tabular"
    unk_freq_threshold: float = 0.0
    block_size: int = 64             # what is the maximum context length for predictions?
    # n_layer: int = 6
    # n_head: int = 6
    # n_embd: int = 384
    # pos_encoding: str = "index-embedding"                 # Manually adding later
    # bias: bool = True
    # attention_type: str = "global"    
    # dropout: float = 0.0

config = DemoConfig()

@dataclass
class OptConfig:
    batch_size: int = 64
    # eval_interval: int = 1
    # learning_rate: float = 3e-4
    # epochs: int = 10
    
opt = OptConfig()

# Data processing

Data is first extracted from CPRD using [DExtER](https://link.springer.com/article/10.1007/s10654-020-00677-6) and is available within the optimal project master dataset. Some outlier filtering has already been done by DExtER in this extraction, and the ICD diagnostic codes in CPRD have been summarised and processed.

In [7]:
print(f"The data can be found here:")
!ls --color -lah /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data

print(f"\n... with baseline covariates stored here")
!ls --color -lah /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/baseline

print(f"\n... and time series events stored here")
!ls --color -lah /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/timeseries

The data can be found here:
total 70K
drwx--S--- 7 nobody nobody  16K Nov 29 10:15 .
drwx--S--- 8 nobody nobody 4.0K Nov 29 10:16 ..
drwx--S--- 2 nobody nobody 4.0K Nov 14 10:44 baseline
-rwx------ 1 nobody nobody 4.0K Oct 23 08:26 ._.DS_Store
-rwx------ 1 nobody nobody 6.1K Nov  4 13:01 .DS_Store
drwx--S--- 3 nobody nobody 4.0K Nov 29 10:15 HES
drwx--S--- 2 nobody nobody 4.0K Feb  8 14:22 metadata
drwx--S--- 4 nobody nobody 4.0K Jan 10 11:04 timeseries
drwx--S--- 3 nobody nobody 4.0K Feb 21 13:30 zip

... with baseline covariates stored here
total 30G
drwx--S--- 2 nobody nobody 4.0K Nov 14 10:44 .
drwx--S--- 7 nobody nobody  16K Nov 29 10:15 ..
-rwx------ 1 nobody nobody  15G Nov 10 15:43 masterDataOptimal_v3.csv
-rwx------ 1 nobody nobody  15G Nov 14 10:27 masterDataOptimalWithIMD_v3.csv

... and time series events stored here
total 98K
drwx--S--- 4 nobody nobody 4.0K Jan 10 11:04 .
drwx--S--- 7 nobody nobody  16K Nov 29 10:15 ..
-rwx------ 1 nobody nobody 4.0K Nov  4 12:57 ._.DS_Sto

# Compressing DExtER output 

These are pre-processed further in R to obtain three .csv files which are in turn compiled into an SQLite database within the `CPRD.data` module. 

* TODO: Replace R filtering with in-built python script
* TODO: It is future work to combine these steps into DExtER, this will massively reduce the memory requirements.
    * DExtER's output has the benefit of keeping records separate. E.g. if we only care about a subset of measurements then we only need to look at one file. However, we cam equally be achieved by a DB query.

In [8]:
print(f"The scripts for compressing these outputs can be found here")
!ls --color -lah /rds/projects/s/subramaa-mum-predict/CharlesGadd_Oxford/FoundationModel/preprocessing

print(f"\n... and the saved files can be found here")
!ls --color -lah /rds/projects/s/subramaa-mum-predict/CharlesGadd_Oxford/FoundationModel/preprocessing/processed

The scripts for compressing these outputs can be found here
total 67K
drwx--S---  3 gaddcz nobody 4.0K Sep 28 11:12 .
drwx--S---. 5 gaddcz nobody 4.0K Oct 24 11:21 ..
drwx--S---  2 gaddcz nobody  16K Oct 18 15:48 processed
-rwx------  1 gaddcz nobody 7.5K Sep 28 15:22 processing_diagnoses.R
-rwx------. 1 gaddcz nobody  14K Oct 18 16:10 processing_measurements_and_tests.R
-rwx------  1 gaddcz nobody 2.2K Sep 19 11:00 processing_static.R

... and the saved files can be found here
total 2.0G
drwx--S--- 2 gaddcz nobody  16K Oct 18 15:48 .
drwx--S--- 3 gaddcz nobody 4.0K Sep 28 11:12 ..
-rwx------ 1 gaddcz nobody 983M Oct 18 15:48 cprd.db
-rwx------ 1 gaddcz nobody 128M Oct  2 15:06 diagnosis_history.csv
-rwx------ 1 gaddcz nobody 874M Sep 28 15:38 measurements.csv
-rwx------ 1 gaddcz nobody  42M Sep 20 12:06 static.csv


# SQL tables

We can convert each of the processed files into separate SQL tables. 

* TODO: Data should never have really left an SQL form here.

## Static table
<div align="justify">
The static table has one row per data owner. 

* PRACTICE_PATIENT_ID: The data owner identifier,
* [ETHNICITY, SEX, YEAR_OF_BIRTH]: The static variables which remain constant throughout a lifetime.
* INDEX_AGE:
* START_AGE:
*  END_AGE: 
</div>

<div align="center">
  TODO: 
</div>

* TODO: Repace missing with nans or some other unique value so these can be masked later
* TODO: Add filtering to only include events after index age



In [5]:
print(f"`Static table` docstring:\n{CPRD.data.database.build_static_db.build_static_table.__doc__}")

`Static table` docstring:

    
    Produced anonymized table:
    ┌──────────────────────┬─────┬───────────┬───────────────┬─────────────┬─────────────┬───────────────┐
    │ PRACTICE_PATIENT_ID  ┆ SEX ┆ ETHNICITY ┆ YEAR_OF_BIRTH ┆ INDEX_AGE   ┆ START_AGE   ┆ END_AGE       │
    │ ---                  ┆ --- ┆ ---       ┆ ---           ┆ ---         ┆ ---         ┆ ---           │
    │ str                  ┆ str ┆ str       ┆ str           ┆ i64 (days)  ┆ i64 (days)  ┆ i64 (days)    │
    ╞══════════════════════╪═════╪═══════════╪═══════════════╪═════════════╪═════════════╪═══════════════╡
    │ <anonymous 1>        ┆ M   ┆ WHITE     ┆ yyyy--mm-dd   ┆ dd          ┆ dd          ┆ dd            │
    │ <anonymous 2>        ┆ F   ┆ MISSING   ┆ yyyy--mm-dd   ┆ dd          ┆ dd          ┆ dd            │
    │ …                    ┆ …   ┆ …         ┆ …             ┆             ┆             ┆               │
    │ <anonymous N>        ┆ M   ┆ WHITE     ┆ yyyy--mm-dd   ┆ dd          ┆ dd  

## Diagnosis table
<div align="justify">
The diagnosis table has one row per diagnosis. 

* PRACTICE_PATIENT_ID: The data owner identifier,
* EVENT: The categorical event
* AGE_AT_EVENT: Number if days between subject birth and the day of event
* VALUE: to be removed
</div>


<div align="center">
  TODO: 
</div>

* We could record based on ICD code subgroups. For example each of these conditions are categorical, but they could be further sub categories. For example, diabetes can be one category and types further divide this.
* remove EVENT_TYPE


In [6]:
print(f"`Diagnosis table` docstring:\n{CPRD.data.database.build_diagnosis_db.build_diagnosis_table.__doc__}")

`Diagnosis table` docstring:

    Build measurements and tests table in database

    Produced anonymized table:
    ┌──────────────────────┬───────┬──────────────┬──────────────┬────────────────────────────┐
    │ PRACTICE_PATIENT_ID  ┆ VALUE ┆ EVENT        ┆ AGE_AT_EVENT ┆ EVENT_TYPE                 │
    │ ---                  ┆ ---   ┆ ---          ┆ ---          ┆ ---                        │
    │ str                  ┆ f64   ┆ str          ┆ i64 (days)   ┆ str                        │
    ╞══════════════════════╪═══════╪══════════════╪══════════════╪════════════════════════════╡
    │ <anonymous 1>        ┆ null  ┆ HF           ┆ 11632        ┆ categorical                │
    │ <anonymous 2>        ┆ null  ┆ HF           ┆ 25635        ┆ categorical                │
    │ …                    ┆ …     ┆ …            ┆ …            ┆ …                          │
    │ <anonymous N>        ┆ null  ┆ FIBROMYALGIA ┆ 8546         ┆ categorical                │
    └──────────────────

## Measurements and tests table
<div align="justify">
The measurements table has one row per diagnosis. 

* PRACTICE_PATIENT_ID: The data owner identifier,
* EVENT: The categorical event
* AGE_AT_EVENT: Number if days between subject birth and the day of event
* VALUE: The measurement/test record
* EVENT_TYPE: whether the value is categorical (e.g. EVENT=="smoking", value="ex-smoker"), or continuous (e.g. EVENT=="bmi", value=23.3)
</div>


<div align="center">
  TODO: 
</div>



In [7]:
print(f"`Measurements and tests table` docstring:\n{CPRD.data.database.build_measurements_and_tests_db.build_measurements_table.__doc__}")

`Measurements and tests table` docstring:
 
    Build measurements and tests table in database

    Produced anonymized table:
    ┌──────────────────────┬───────┬──────────────────┬──────────────┬───────────────────────┐
    │ PRACTICE_PATIENT_ID  ┆ VALUE ┆ EVENT            ┆ AGE_AT_EVENT ┆ EVENT_TYPE            │
    │ ---                  ┆ ---   ┆ ---              ┆ ---          ┆ ---                   │
    │ str                  ┆ f64   ┆ str              ┆ i64 (days)   ┆ str                   │
    ╞══════════════════════╪═══════╪══════════════════╪══════════════╪═══════════════════════╡
    │ <anonymous 1>        ┆ 23.3  ┆ bmi              ┆ 10254        ┆ univariate_regression │
    │ <anonymous 1>        ┆ 24.1  ┆ bmi              ┆ 11829        ┆ univariate_regression │
    │ …                    ┆ …     ┆ …                ┆ …            ┆ …                     │
    │ <anonymous N>        ┆ 0.17  ┆ eosinophil_count ┆ 12016        ┆ univariate_regression │
    └─────────────

# Filtering

We can query the SQL tables separately to find the set of patients that fit the study criteria. 

First we can connect directly to the database using sqlite3

<div align="center">
  TODO: 
</div>

* The db shouldn't really be interfaced with like this, this is mostly for demonstration and a 

In [9]:
PATH_TO_DB = "/rds/projects/s/subramaa-mum-predict/CharlesGadd_Oxford/FoundationModel/preprocessing/processed/cprd.db"
conn = sqlite3.connect(PATH_TO_DB)
cursor = conn.cursor()

Performing a look-up directly on the tables shows what diagnoses, measurements and tests are available

In [10]:
# Check what measurements are available
cursor.execute("SELECT DISTINCT EVENT FROM measurement_table")
measurements = [_item[0] for _item in cursor.fetchall()]
print("We have access to measurements and tests:\n * " + '\n * '.join(measurements))

# Check what diagnoses are available
cursor.execute("SELECT DISTINCT EVENT FROM diagnosis_table")
diagnoses = [_item[0] for _item in cursor.fetchall()]
print("\nAdditionally, we have access to diagnoses:\n * " + '\n * '.join(diagnoses))

We have access to measurements and tests:
 * bmi
 * hydroxyvitamin2
 * hydroxyvitamin3
 * aspartate_transam
 * serum_level
 * creatinine_ratio
 * basophil_count
 * blood_calcium
 * blood_urea
 * brain_natriuretic_peptide_level
 * calcium_adjusted_level
 * calculated_LDL_cholesterol_level
 * combined_total_vitamin_D2_and_D3_level
 * corrected_serum_calcium_level
 * diastolic_blood_pressure
 * eosinophil_count

Additionally, we have access to diagnoses:
 * HF
 * AF
 * ISCHAEMICSTROKE
 * STROKEUNSPECIFIED
 * STROKE_HAEMRGIC
 * HYPERTENSION
 * MINFARCTION
 * IHD_NOMI
 * PAD_STRICT
 * VALVULARDISEASES
 * AORTICANEURYSM
 * TYPE1DM
 * TYPE2DIABETES
 * CKDSTAGE3TO5
 * DEPRESSION
 * ANXIETY
 * BIPOLAR
 * EATINGDISORDERS
 * SCHIZOPHRENIAMM
 * AUTISM
 * ALCOHOLMISUSE
 * SUBSTANCEMISUSE
 * CHRONIC_LIVER_DISEASE_ALCOHOL
 * NAFLD
 * OTHER_CHRONIC_LIVER_DISEASE_OPTIMAL
 * ULCERATIVE_COLITIS
 * CROHNS_DISEASE
 * ALL_DEMENTIA
 * PARKINSONS
 * EPILEPSY
 * ALLCA_NOBCC_VFINAL
 * LYMPHOMA_PREVALENCE
 * LEU

Given some subset of these we can query the database to find those subjects who have at least one entry of a list of measurements, tests or diagnoses

In [12]:
identifiers1 = CPRD.data.database.queries.query_measurement(["bmi", "diastolic_blood_pressure"], cursor)        
identifiers2 = CPRD.data.database.queries.query_diagnosis(["DEPRESSION", "TYPE1DM", "TYPE2DIABETES"], cursor)    #  "DEPRESSION"  ,  "ANXIETY"
all_identifiers = list(set(identifiers1).intersection(identifiers2))    # Turn smaller list into the set

print(all_identifiers[:10])

['p20508_942381720508', 'p20524_2116546320524', 'p20433_213995120433', 'p20655_1198195820655', 'p20433_213498220433', 'p20491_964962720491', 'p20524_2116378620524', 'p20679_996260920679', 'p20692_5495714120692', 'p20741_1100618520741']


In [15]:
# For now, lets take only a random subset of 10,000
N = np.min((len(all_identifiers), 10000))
print(f"Using N={N} random samples, from the available {len(all_identifiers)} with satisfied our criteria")

identifiers = random.choices(all_identifiers, k=N)

Using N=10000 random samples, from the available 117102 with satisfied our criteria


# Make PyTorch dataloader

We can now initialise our DL friendly datasets. This is done within the DataModule initialisation. 

Initialisation takes one required argument, which is the set of patient `identifiers` we wish to build into our dataset. The remaining arguments are optional and explained in the docstring


<div align="center">
  TODO: 
</div>

* Add option of saving and loading initialised train/test/val splits

In [12]:
print(f"`FoundationalDataModule` docstring:\n{FoundationalDataModule.__doc__}")

`FoundationalDataModule` docstring:

    PyTorch-Lightning datamodule for foundational models
    
    ARGS:
        practice_patient_id (list[str])
            List of practice patient identifiers which satisfy study criteria.
            
    KWARGS:
        batch_size (int): 
        
        unk_freq_threshold (float). 
            Value between 0 and 1, controlling at what level of frequency rare tokens (equiv. conditions/measurements 
            with this tokenizer) are mapped to the UNK token. Used to reduce vocabulary size
            
        min_workers (int):

        load_event_stream (optional, str):
        
        save_event_stream (optional, str):
        
    


In [19]:
dm = FoundationalDataModule(identifiers=identifiers,
                            tokenizer=config.tokenizer,
                            batch_size=opt.batch_size,
                            max_seq_length=config.block_size,
                            unk_freq_threshold=config.unk_freq_threshold,
                            include_measurements=True,
                            include_diagnoses=True,
                            preprocess_measurements=False
                            )

print(f"{len(dm.train_set)} training samples")
print(f"{len(dm.val_set)} validation samples")
print(f"{len(dm.test_set)} test samples")
print(f"{dm.tokenizer.vocab_size} dictionary elements")

INFO:root:Building polars dataset
INFO:root:Using measurements
INFO:root:Using diagnoses
INFO:root:Dropping samples with no dynamic events
INFO:root:Using tabular tokenizer


8633 training samples
480 validation samples
480 test samples
89 dictionary elements


## Tokenizer 

This creates a tokenizer that is shared across each train/test/val split.

The tokenizer can either be "tabular", in which values tied to events are treated separately, and loaded in batches under the key `"values"`

In [28]:
dm = FoundationalDataModule(identifiers=identifiers,
                            tokenizer="tabular",
                            batch_size=opt.batch_size,
                            max_seq_length=config.block_size,
                            unk_freq_threshold=config.unk_freq_threshold,
                            include_measurements=True,
                            include_diagnoses=True,
                            preprocess_measurements=False
                            )

display(list(dm.tokenizer._stoi.keys())[:10])

for batch in dm.train_dataloader():
    break
display(batch["values"])

INFO:root:Building polars dataset
INFO:root:Using measurements
INFO:root:Using diagnoses
INFO:root:Dropping samples with no dynamic events
INFO:root:Using tabular tokenizer


['PAD',
 'UNK',
 'diastolic_blood_pressure',
 'bmi',
 'eosinophil_count',
 'basophil_count',
 'corrected_serum_calcium_level',
 'DEPRESSION',
 'serum_level',
 'calculated_LDL_cholesterol_level']

tensor([[     nan,      nan,  80.0000,  ...,  29.5000,  78.0000,  28.7000],
        [100.0000,      nan,  80.0000,  ...,  70.0000,  72.0000,  26.6000],
        [ 79.0000,  78.0000,  41.6000,  ...,   0.2900,  73.0000,  36.5000],
        ...,
        [     nan,      nan,      nan,  ...,      nan,      nan,      nan],
        [ 80.0000,  84.0000,  82.0000,  ...,      nan,      nan,      nan],
        [ 36.0000,  87.0000,  37.6000,  ...,      nan,      nan,      nan]])

The tokenizer also be `"non-tabular"`, in which values are concatenated onto events.

For example, as in https://proceedings.neurips.cc/paper_files/paper/2023/file/3eb7ca52e8207697361b2c0fb3926511-Paper-Conference.pdf

In [30]:
dm = FoundationalDataModule(identifiers=identifiers,
                            tokenizer="non-tabular",
                            batch_size=opt.batch_size,
                            max_seq_length=config.block_size,
                            unk_freq_threshold=config.unk_freq_threshold,
                            include_measurements=True,
                            include_diagnoses=True,
                            preprocess_measurements=False
                            )

display(list(dm.tokenizer._stoi.keys())[:10])

for batch in dm.train_dataloader():
    break
display(batch["values"])

INFO:root:Building polars dataset
INFO:root:Using measurements
INFO:root:Using diagnoses
INFO:root:Dropping samples with no dynamic events
INFO:root:Using non-tabular tokenizer


['PAD', 'UNK', '0', '1', '2', '3', '4', '5', '6', '7']

tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]])

## Standardisation and pre-processing

We can also perform standardisation and pre-processing on creating the dataloader. Note, DExtER already performs outlier removal, but I found it beneficial to repeat this.

Normalisation values are stored in a dictionary so the inverse transformation can be performed on any model outputs.

This also works for the `"non-tabular"` tokenizer, but we do not truncate significant figures.

In [38]:
dm = FoundationalDataModule(identifiers=identifiers,
                            tokenizer="tabular",
                            batch_size=opt.batch_size,
                            max_seq_length=config.block_size,
                            unk_freq_threshold=config.unk_freq_threshold,
                            include_measurements=True,
                            include_diagnoses=True,
                            preprocess_measurements=True
                            )

display(dm.standardisation_dict)

INFO:root:Building polars dataset
INFO:root:Using measurements
INFO:root:Using test/measurement standardisation method: normalise
INFO:root:Removing measurement and test outliers. Using three deviations from mean as cutoff
INFO:root:Using diagnoses
INFO:root:Dropping samples with no dynamic events
INFO:root:Using tabular tokenizer


{'blood_calcium': (2.3055555555555554, 0.09426010460561489),
 'hydroxyvitamin3': (50.486464088397774, 28.253421025622043),
 'serum_level': (27.36, 20.39314546302366),
 'blood_urea': (6.433777506112473, 3.5687089758899444),
 'basophil_count': (0.06952197006276979, 0.0995862931307564),
 'combined_total_vitamin_D2_and_D3_level': (55.69924812030075,
  28.274832916537175),
 'bmi': (29.81121446763248, 7.217455651019206),
 'calculated_LDL_cholesterol_level': (2.6379761713089747, 1.044809612939309),
 'aspartate_transam': (26.781417112299465, 19.880841072154606),
 'corrected_serum_calcium_level': (2.3245091185410347, 0.12969810940207963),
 'brain_natriuretic_peptide_level': (100.22884615384615, 155.17429964150236),
 'creatinine_ratio': (4.052689655172414, 7.437664867936673),
 'diastolic_blood_pressure': (78.92946571661037, 11.728599302849629),
 'eosinophil_count': (0.22289105762382594, 0.18930721183486415),
 'hydroxyvitamin2': (3.4080597014925367, 2.913374198070332),
 'calcium_adjusted_level': 

## The inner workings for the Datamodule: The Dataset

Each dataset within the DataModule is a ``cprd.data.dataset.dataset_polars.EventStreamDataset`` object. Within this class we process the SQL tables from a form in which each patient (static) or record (dianosis/measurement/test) has its own row, into a ragged form where each patient has their own row in every case. This does not change the form of the static table. However, the form of the diagnoses and measurements/tests changes - and during the same update we combine these frames to one frame which contains dynamic information.

This is all wrapped in the method ``fit()``, but we can see the separate steps below:

In [15]:
print(f"`_load_dynamic` docstring:\n{CPRD.data.dataset.dataset_polars.EventStreamDataset._load_dynamic.__doc__}")

`_load_dynamic` docstring:
    
        Load and merge dynamic tables from SQL
        
        ARGS:
            practice_patient_id (list[str])
                List of practice patient identifiers which satisfy study criteria.
            
        KWARGS:
            include_measurements: bool = True,
                Flag of whether to include measurements and tests in polars dataframe
            include_diagnoses: bool = True
                Flag of whether to include diagnoses in polars dataframe
        
        RETURNS:
            Polars lazy frame, of the form:
            ┌──────────────────────┬───────────────────────┬───────────────────────────────────┬─────────────────────────┬───────────────────────────────────┐
            │ PRACTICE_PATIENT_ID  ┆ VALUE                 ┆ EVENT                             ┆ AGE_AT_EVENT            ┆ EVENT_TYPE                        │
            │ ---                  ┆ ---                   ┆ ---                               ┆ ---     

This is then combined into one DL friendly representation, which our DataModule uses

In [16]:
print(f"`_build_DL_representation` docstring:\n{CPRD.data.dataset.dataset_polars.EventStreamDataset._build_DL_representation.__doc__}")

`_build_DL_representation` docstring:

        Build the DL-friendly representation in polars given the list of `practice_patient_id`s which fit study criteria
                
        ARGS:
            practice_patient_ids (list[str])
                List of practice patient identifiers which satisfy study criteria.
            
        KWARGS:
        
        
        RETURNS:
            Polars lazy frame, of the (anonymized) form:
            ┌──────────────────────┬─────┬─────────────┬───────────────┬──────────────────────┬─────────────────────────┬─────────────────────┬────────────────────────┐
            │ PRACTICE_PATIENT_ID  ┆ SEX ┆ ETHNICITY   ┆ YEAR_OF_BIRTH ┆ VALUE                ┆ EVENT                   ┆ AGE_AT_EVENT        ┆ EVENT_TYPE             │
            │ ---                  ┆ --- ┆ ---         ┆ ---           ┆ ---                  ┆ ---                     ┆ ---                 ┆ ---                    │
            │ str                  ┆ str ┆ str       

The ``__getitem__`` within the dataset class then retrieves rows of this table and performs the relevant processing such as tokenization. Speed ups could be obtained by pre-processing this tokenization.

As these are ragged lists for memory efficiency, we also provide a ``collate_fn`` in the DataModule. This performs padding - and by performing padding this way we can avoid excessive padding.

# Data iterators

## Using \_\_getitem__

In [39]:
# print(dm.train_set[0])
print("A single element of the training dataset contains:\n  * " + '\n  * '.join(dm.train_set[0].keys()))

sample = np.random.randint(len(dm.train_set))

for k, v in dm.train_set[sample].items():
    print(f"\n{k}: {v}")
    if k == "tokens":
        print(f"... decoding to `{dm.decode(v.tolist())}`")


A single element of the training dataset contains:
  * identifier
  * tokens
  * ages
  * values

identifier: p20684_2602734120684

tokens: tensor([13, 10,  2,  7,  3,  4,  3,  2,  2])
... decoding to `ASTHMA_PUSHASTHMA ANXIETY diastolic_blood_pressure DEPRESSION bmi eosinophil_count bmi diastolic_blood_pressure diastolic_blood_pressure`

ages: tensor([ 107, 5247, 5415, 5696, 5821, 6143, 7184, 7184, 8465])

values: tensor([    nan,     nan, -0.3350,     nan, -1.5533, -0.6492, -0.9714, -0.7613,
         0.0913])


## Collating into batches

In [40]:
for idx, batch in enumerate(dm.train_dataloader()):
    break
print("A sample from the dataloader batch gives:\n  * " + '\n  * '.join(batch.keys()))

k = 10
print(f"\nFor example, a single sample from a collated batch gives (viewing only first {k} elements of each padded sequence):")

# ages
print(f"\n* The time of event (in days since birth) of events:\n\t {batch['ages'][0, :k]}")

# tokens
print(f"\n* The tokenized and padded (within batch) block of a patient's sequence of events:\n\t {batch['tokens'][0,:k]}")
print(f"... which can be decoded. E.g. the above tokens decode to:\n\t {dm.decode(batch['tokens'][0,:k].tolist())}`")

# values
print(f"\n* When we use a 'tabular' tokenizer, the corresponding values are given as:\n\t {batch['values'][0,:k]}")

# attention mask
#   B = batch size, L = Longest sequence length within batch
print(f"\n* The attention mask tells us which values are padded entries, so we can then mask them alongside the self-attention masking" + 
      f"\n\t This is of shape (B={batch['attention_mask'].shape[0]} * L={batch['attention_mask'].shape[1]})" +
      f"\n{batch['attention_mask']}")



A sample from the dataloader batch gives:
  * tokens
  * ages
  * values
  * attention_mask

For example, a single sample from a collated batch gives (viewing only first 10 elements of each padded sequence):

* The time of event (in days since birth) of events:
	 tensor([4141, 8894, 8894, 8902, 8902, 8917, 8917, 9110, 9266, 9397])

* The tokenized and padded (within batch) block of a patient's sequence of events:
	 tensor([19,  5,  4,  5,  4,  5,  4,  2,  2,  3])
... which can be decoded. E.g. the above tokens decode to:
	 PREVALENT_IBS basophil_count eosinophil_count basophil_count eosinophil_count basophil_count eosinophil_count diastolic_blood_pressure diastolic_blood_pressure bmi`

* When we use a 'tabular' tokenizer, the corresponding values are given as:
	 tensor([    nan, -0.5977, -0.3322, -0.4973, -0.3322, -0.5977, -0.3322,  0.6028,
        -0.5908, -0.1581])

* The attention mask tells us which values are padded entries, so we can then mask them alongside the self-attention ma

# Tabular vs. non-tabular tokenization

We can also use a non-tabular tokenizer.

In [35]:
dm = FoundationalDataModule(identifiers=identifiers,
                            tokenizer="non-tabular",
                            batch_size=opt.batch_size,
                            max_seq_length=config.block_size,
                            unk_freq_threshold=config.unk_freq_threshold)

print(f"{len(dm.train_set)} training samples")
print(f"{len(dm.val_set)} validation samples")
print(f"{len(dm.test_set)} test samples")
print(f"{dm.tokenizer.vocab_size} dictionary elements")

print("\nDictionaries")
print(dm.tokenizer._itos)
# print(dm.tokenizer._stoi)

INFO:root:Building polars dataset
INFO:root:Using measurements
INFO:root:Using test/measurement standardisation method: normalise
INFO:root:Removing measurement and test outliers. Using three deviations from mean as cutoff
INFO:root:Using diagnoses
INFO:root:Dropping samples with no dynamic events
INFO:root:Using non-tabular tokenizer


8603 training samples
478 validation samples
478 test samples
101 dictionary elements

Dictionaries
{0: 'PAD', 1: 'UNK', 2: '0', 3: '1', 4: '2', 5: '3', 6: '4', 7: '5', 8: '6', 9: '7', 10: '8', 11: '9', 12: '.', 13: 'diastolic_blood_pressure', 14: 'bmi', 15: 'eosinophil_count', 16: 'basophil_count', 17: 'corrected_serum_calcium_level', 18: 'DEPRESSION', 19: 'serum_level', 20: 'calculated_LDL_cholesterol_level', 21: 'ANXIETY', 22: 'HYPERTENSION', 23: 'TYPE2DIABETES', 24: 'OSTEOARTHRITIS', 25: 'ASTHMA_PUSHASTHMA', 26: 'ATOPICECZEMA', 27: 'ALLERGICRHINITISCONJ', 28: 'aspartate_transam', 29: 'ANY_DEAFNESS_HEARING_LOSS', 30: 'PREVALENT_IBS', 31: 'IHD_NOMI', 32: 'ALCOHOLMISUSE', 33: 'ALLCA_NOBCC_VFINAL', 34: 'CKDSTAGE3TO5', 35: 'blood_urea', 36: 'PERIPHERAL_NEUROPATHY', 37: 'HYPOTHYROIDISM_DRAFT_V1', 38: 'calcium_adjusted_level', 39: 'COPD', 40: 'combined_total_vitamin_D2_and_D3_level', 41: 'AF', 42: 'PSORIASIS', 43: 'HF', 44: 'GOUT', 45: 'OSTEOPOROSIS', 46: 'SUBSTANCEMISUSE', 47: 'MINFARCTI

In [37]:
for idx, batch in enumerate(dm.train_dataloader()):
    break
print("A sample from the dataloader batch gives:\n  * " + '\n  * '.join(batch.keys()))

k = 10
print(f"\nRepeating the above example, a single sample from a collated batch gives (viewing only first {k} elements of each sequence):")

# ages
print(f"\n* The time of event (in days since birth) of events:\n\t {batch['ages'][0, :k]}")

# tokens
print(f"\n* The tokenized and padded (within batch) block of a patient's sequence of events:\n\t {batch['tokens'][0,:k]}")
print(f"... which can be decoded. E.g. the above tokens decode to:\n\t {dm.decode(batch['tokens'][0,:k].tolist())}`")

# values
print(f"\n* When we use a {config.tokenizer} tokenizer, the corresponding values are given as:\n\t {batch['values'][0,:k]}")

# attention mask
#   B = batch size, L = Longest sequence length within batch
print(f"\n* The attention mask tells us which values are padded entries, so we can then mask them alongside the self-attention masking" + 
      f"\n\t This is of shape (B={batch['attention_mask'].shape[0]} * L={batch['attention_mask'].shape[1]})" +
      f"\n{batch['attention_mask']}")



A sample from the dataloader batch gives:
  * tokens
  * ages
  * values
  * attention_mask

Repeating the above example, a single sample from a collated batch gives (viewing only first 10 elements of each sequence):

* The time of event (in days since birth) of events:
	 tensor([7004, 7009, 7009, 7009, 7009, 7009, 7009, 7009, 7009, 7009])

* The tokenized and padded (within batch) block of a patient's sequence of events:
	 tensor([ 6, 15,  2, 12,  4, 11,  8,  4,  2,  6])
... which can be decoded. E.g. the above tokens decode to:
	 4 eosinophil_count 0 . 2 9 6 2 0 4`

* When we use a tabular tokenizer, the corresponding values are given as:
	 tensor([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan])

* The attention mask tells us which values are padded entries, so we can then mask them alongside the self-attention masking
	 This is of shape (B=64 * L=64)
tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1